In [38]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [39]:
options = uc.ChromeOptions()
url = 'https://tixcraft.com/activity/detail/23_abc30'
driver = uc.Chrome()
driver.get(url)

In [41]:
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[href='#login']"))).click()

In [42]:
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[id='loginGoogle']"))).click()


In [48]:
driver.find_element(By.CSS_SELECTOR, "input[class='whsOnd zHQkBf']").send_keys('tnecniv43208')
driver.find_element(By.CSS_SELECTOR, "span[class='VfPpkd-RLmnJb']").click()

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=119.0.6045.199)
Stacktrace:
0   undetected_chromedriver             0x000000010278ed28 undetected_chromedriver + 4795688
1   undetected_chromedriver             0x00000001027862b3 undetected_chromedriver + 4760243
2   undetected_chromedriver             0x000000010235f6bd undetected_chromedriver + 407229
3   undetected_chromedriver             0x00000001023afb43 undetected_chromedriver + 736067
4   undetected_chromedriver             0x00000001023a23fa undetected_chromedriver + 680954
5   undetected_chromedriver             0x00000001023d5ca2 undetected_chromedriver + 892066
6   undetected_chromedriver             0x00000001023a1c63 undetected_chromedriver + 679011
7   undetected_chromedriver             0x00000001023d5e5e undetected_chromedriver + 892510
8   undetected_chromedriver             0x00000001023f1b2b undetected_chromedriver + 1006379
9   undetected_chromedriver             0x00000001